<a href="https://colab.research.google.com/github/leejunho12316/Machine-Learning/blob/main/%EB%8B%A4%EC%A4%91%EC%B6%9C%EB%A0%A5/%EB%8B%A4%EC%A4%91%EC%B6%9C%EB%A0%A5_%EC%97%B0%EC%8A%B51.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://tensorflow.blog/%EC%BC%80%EB%9D%BC%EC%8A%A4-%EB%94%A5%EB%9F%AC%EB%8B%9D/3-5-%EB%89%B4%EC%8A%A4-%EA%B8%B0%EC%82%AC-%EB%B6%84%EB%A5%98-%EB%8B%A4%EC%A4%91-%EB%B6%84%EB%A5%98-%EB%AC%B8%EC%A0%9C/
## **단일 레이블 다중 분류 - 각 데이터포인트가 여러개의 범주에 속할 수 있음**
다중 레이블 다중 분류 - 각 데이터포인트가 여러개의 범주에 속할 수 있음
ctrl m h

## **1. 데이터 준비**
keras 에서 reuter 기사 데이터 가져오기. 데이터는 이미 토큰화->시퀀스변환 과정을 마친 상태로 불러와진다

In [ ]:
from keras.datasets import reuters

(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000) #num_words : 데이터에서 자주 등장하는 단어 상위 10000개로 개수 제한

2110848/2110848 [==============================] - 0s 0us/step


In [ ]:
train_labels

array([ 3,  4,  3, ..., 25,  3, 25])

각 범주별로 몇 개의 데이터가 있는지

In [ ]:
total_labels = {}
for i in train_labels:
  total_labels[i] = total_labels.get(i,0)+1

sorted_dict = {key:total_labels[key] for key in sorted(total_labels)}
print(sorted_dict)

{0: 55, 1: 432, 2: 74, 3: 3159, 4: 1949, 5: 17, 6: 48, 7: 16, 8: 139, 9: 101, 10: 124, 11: 390, 12: 49, 13: 172, 14: 26, 15: 20, 16: 444, 17: 39, 18: 66, 19: 549, 20: 269, 21: 100, 22: 15, 23: 41, 24: 62, 25: 92, 26: 24, 27: 15, 28: 48, 29: 19, 30: 45, 31: 39, 32: 32, 33: 11, 34: 50, 35: 10, 36: 49, 37: 19, 38: 19, 39: 24, 40: 36, 41: 30, 42: 13, 43: 21, 44: 12, 45: 18}


데이터 봐보기

In [ ]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value,key) for (key,value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i-3,'?') for i in train_data[10]])
decoded_newswire

550378/550378 [==============================] - 0s 1us/step


'? period ended december 31 shr profit 11 cts vs loss 24 cts net profit 224 271 vs loss 511 349 revs 7 258 688 vs 7 200 349 reuter 3'

### **데이터 준비**

In [ ]:
#데이터 벡터변환 (원-핫 인코딩 방식 벡터변환으로 각 기사에 어떤 단어가 사용되었는지에 대한 정보를 담고 있다.)
import numpy as np

#행 : 각 기사들, 열 : 단어 시퀀스, 값 : 기사의 해당 단어 사용 여부
def vectorize_squences(sequences, dimension=10000):
  results = np.zeros((len(sequences),dimension))
  print(results.shape)

  #i번째 원소의 기사 sequence를 np.zero 각각의 줄에 그냥 다 넣기
  for i, sequence in enumerate(sequences):
    results[i,sequence] = 1
  return results

x_train = vectorize_squences(train_data)
x_test = vectorize_squences(test_data)

(8982, 10000)
(2246, 10000)


In [ ]:
for i,seq in enumerate(train_data):
  print(i, seq)
  print()

In [ ]:
#데이터 원-핫 인코딩 벡터변환 ()
def to_one_hot(labels, dimension=46):
  results = np.zeros((len(labels),dimension))
  for i, label in enumerate(labels):
    results[i,label] = 1
  return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

In [ ]:
one_hot_train_labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

데이터 나누기

In [ ]:
#시퀀스-벡터 한 데이터 1:9로 나누기
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

#label one_hot_encoding 한 데이터 1:9로 나누기
y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

## 모델 구성
출력 클래스의 개수가 2개가 아니라 46개입니다

In [ ]:
from keras import models
from keras import layers

#Sequential 모델 정의
model = models.Sequential()
#입력층 - 64개의 뉴런, relu 활성화 함수, 10000개의 입력데이터
model.add(layers.Dense(64, activation='relu',input_shape=(10000,)))
#은닉층
model.add(layers.Dense(64, activation='relu'))
#출력층
model.add(layers.Dense(46, activation='softmax'))


model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

훈련

In [ ]:
#keras Sequential model에 나눈 데이터 중 9쪽 데이터를 넣어 train하고 validation으로는 1쪽을 넣음
history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=20,
                    batch_size=512,
                    validation_data=(x_val,y_val))

Epoch 1/20
16/16 [==============================] - 2s 64ms/step - loss: 2.7583 - accuracy: 0.5127 - val_loss: 1.8137 - val_accuracy: 0.6190
Epoch 2/20
16/16 [==============================] - 1s 43ms/step - loss: 1.5217 - accuracy: 0.6817 - val_loss: 1.3636 - val_accuracy: 0.6980
Epoch 3/20
16/16 [==============================] - 1s 41ms/step - loss: 1.1779 - accuracy: 0.7486 - val_loss: 1.1938 - val_accuracy: 0.7470
Epoch 4/20
16/16 [==============================] - 1s 72ms/step - loss: 0.9697 - accuracy: 0.7982 - val_loss: 1.1172 - val_accuracy: 0.7500
Epoch 5/20
16/16 [==============================] - 1s 69ms/step - loss: 0.8125 - accuracy: 0.8266 - val_loss: 1.0701 - val_accuracy: 0.7590
Epoch 6/20
16/16 [==============================] - 1s 71ms/step - loss: 0.6810 - accuracy: 0.8524 - val_loss: 0.9868 - val_accuracy: 0.7880
Epoch 7/20
16/16 [==============================] - 1s 47ms/step - loss: 0.5738 - accuracy: 0.8763 - val_loss: 0.9350 - val_accuracy: 0.7940
Epoch 8/20
16

새로운 데이터 예측

In [ ]:
pred = model.predict(x_test)

71/71 [==============================] - 0s 3ms/step


In [ ]:
np.argmax(pred[0])

3